In [1]:
import pandas as pd
import os.path

In [2]:
DATASETS = ['Birds', 'Sky', 'ECSSD', 'Insects', 'NYUV2']

METHODS = ['AINET', 'CRS', 'DISF', 'DRW', 'ERGC', 'ETPS', 'GMMSP', 
           'GRID', 'IBIS', 'ISF', 'LSC', 'RSS', 'SCALP', 
           'SH', 'SICLE', 'SIN', 'SLIC', 'SNIC', 'SSFCN',
           'DAL-HERS', 'ERS', 'LNSNet', 'ODISF', 'SEEDS']

SPXS = [25,50,75,100,200,300,400,500,600,700,800,900,1000]
METRICS = ['BR', 'UE', 'EV', 'SIRS']

OUTPUT_PATH="../../RESULTS/Eval"
INPUT_PATH="../../RESULTS/Eval"

# methods that generate unconnected superpixels, we evaluated stability after enforcing connectivity.
ENFORCED_CONNECTIVITY_METHODS = ['CRS', 'SEEDS', 'LNSNet', 'DRW']
ENFORCED_CONNECTIVITY_INPUT = INPUT_PATH + '/EnforcedConnectivity'
ENFORCED_CONNECTIVITY_OUTPUT = OUTPUT_PATH + '/EnforcedConnectivity'

# To enforce connectivity, execute run_eval_enforceConnectivity.sh or run_eval_manual_enforceConnectivity.sh

In [3]:
for dataset in DATASETS:
    for method in METHODS:
        for metric in METRICS:

          # get min / max / std
          data = pd.DataFrame({'spx' : [], 'min' : [], 'std' : [], 'max' : []})

          # load data
          for num_superpixel in SPXS:

            if method in ENFORCED_CONNECTIVITY_METHODS and metric != 'Connectivity' and metric != 'SuperpixelNumber':
                input_file = ENFORCED_CONNECTIVITY_INPUT + '/' +method+'/'+dataset+'/'+metric+'/'+method+'-'+str(num_superpixel)+'.txt'
            else:
              input_file = INPUT_PATH + '/' +method+'/'+dataset+'/'+metric+'/'+method+'-'+str(num_superpixel)+'.txt'

            data_imgs = pd.read_csv(input_file, sep=" ")
            data_imgs.columns = ["Image", "Superpixels", "Score"]
            
            values = {'spx': data_imgs["Superpixels"].mean(),
                      'min': data_imgs['Score'].min(),
                      'std': data_imgs['Score'].std(),
                      'max': data_imgs['Score'].max()}
            
            data = pd.concat([data, pd.DataFrame({
                      'spx': [data_imgs["Superpixels"].mean()],
                      'min': [data_imgs['Score'].min()],
                      'std': [data_imgs['Score'].std()],
                      'max': [data_imgs['Score'].max()]})], ignore_index=True)

          # save data
          for stab_type in ['min', 'max', 'std']:

            output_file = OUTPUT_PATH + '/stability/'+ stab_type
            
            if not os.path.exists(output_file):
              os.makedirs(output_file)
            output_file = output_file+'/'+method+'-'+dataset+'-'+metric+'.txt'
            
            save = pd.DataFrame({'spx' : data['spx'], stab_type : data[stab_type]})
            save.to_csv(output_file, header=['K', 'VALOR'], index=None, sep=' ', mode='w')